# Demo 2: next steps

In [1]:
from blase import emulator 

In [2]:
emulator.SparseLinearEmulator

blase.emulator.SparseLinearEmulator

In [3]:
emulator.SparseLogEmulator

blase.emulator.SparseLogEmulator